In [12]:
import orjson
pwd = '~/projects/gun-overlays/bible/'

In [13]:
with open(pwd+'/roh.json') as f:
    data = orjson.loads(f.read())

for book in data['books']:
    with open(pwd+'/roh/'+book['abbreviation']+'.json', 'wb') as f:
        f.write(orjson.dumps(book.pop('chapters'), option=orjson.OPT_INDENT_2))

with open(pwd+'/roh/index.json', 'wb') as f:
    f.write(orjson.dumps(data, option=orjson.OPT_INDENT_2))

In [15]:
with open(pwd+'/seb.json') as f:
    data = orjson.loads(f.read())

for book in data['books']:
    with open(pwd+'/seb/'+book['abbreviation']+'.json', 'wb') as f:
        f.write(orjson.dumps(book.pop('chapters'), option=orjson.OPT_INDENT_2))

with open(pwd+'/seb/index.json', 'wb') as f:
    f.write(orjson.dumps(data, option=orjson.OPT_INDENT_2))

# Scrape biblia.sk

In [4]:
from time import time, sleep
import binascii
import json
import requests

res = requests.get('https://api2.biblia.sk/api/preklady')
preklady = res.json()['data']
with open('preklady.json', 'w') as f:
    json.dump(preklady, f, indent=2, ensure_ascii=False)

for identifier in ['bot']:  #'seb','sep','roh','ssv',
    preklad = None
    for p in preklady:
        if p["identifier"] == identifier:
            preklad = p
            break
    if preklad is None:
        print("Identifier not found", identifier)
        continue
    books = preklad['books']

    # params = {
    #     'timestamp': int(time()*1000),
    #     'key': '597133743677397A24432646294A404E635166546A576E5A7234753778214125',
    # }
    for i, book in enumerate(books):
        chapters = {}
        abbr = book['abbreviation']
        for ch in book['chapters']:
            # params['timestamp'] = int(time()*1000)
            # addr = f"{identifier}|{abbr}|{chapter}"
            # code = binascii.b2a_base64(addr.encode()).strip().decode()
            while True:
                res = requests.get(f"https://biblia.sk/api/chapter?translation={identifier}&book={abbr}&chapter={ch}")
                print(res.status_code, res.url[:70])
                if res.status_code == 429:
                    print('Too Many Requests, sleeping')
                    sleep(40)
                    continue
                break
            try:
                text = res.json()
            except Exception as e:
                print(res.content)
                raise e
            chapters[str(ch)] = [v['search_text'].strip() for v in text['data']]
        book['chapters'] = chapters
        # Write book.json
        with open(f"{identifier}/{abbr}.json", "w") as f:
            json.dump(chapters, f, indent=2, ensure_ascii=False)
    break


200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=1
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=2
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=3
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=4
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=5
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=6
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=7
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=8
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=9
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=10
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=11
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=12
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=13
200 https://biblia.sk/api/chapter?translation=bot&book=gn&chapter=14
200 https://biblia.sk/api/chapter?translati

In [8]:
# Create routes/bible/{bible}.json.js
import json
with open('../../src/routes/bible/seb.json.js') as f:
    seb = json.loads(f.read()[21:-66])
sebaliases = {b["abbreviation"]: b["aliases"] for b in seb["books"] if "aliases" in b}
with open('preklady.json') as f:
    preklady = json.load(f)

for identifier in ['ssv', 'bot']:
    preklad = None
    for p in preklady:
        if p["identifier"] == identifier:
            preklad = p
            break
    for book in preklad["books"]:
        book.pop("chapters")
        try:
            book["aliases"] = sebaliases[book["abbreviation"]]
        except KeyError:
            pass
    with open(f'../../src/routes/bible/{identifier}.json.js', 'w') as f:
        f.write('export const index = ')
        json.dump(preklad, f, indent=2, ensure_ascii=False)
        f.write(""";

    export async function get() {
      return {
        body: index,
      }
    }""")


In [11]:
# Correct order of books
with open(f'../../src/routes/bible/seb.json.js') as f:
    seb = json.loads(f.read()[21:-66])

for identifier in ['ssv', 'bot']:
    with open(f'../../src/routes/bible/{identifier}0.json.js') as f:
        bible = json.loads(f.read()[21:-66])
    reordered_books = []
    for book in seb["books"]:
        for b in bible["books"]:
            if b["abbreviation"] == book["abbreviation"]:
                reordered_books.append(b)
                break
    bible["books"] = reordered_books
    with open(f'../../src/routes/bible/{identifier}.json.js', 'w') as f:
        f.write('export const index = ')
        json.dump(bible, f, indent=2, ensure_ascii=False)
        f.write(""";

    export async function get() {
      return {
        body: index,
      }
    }""")

In [19]:
# Correct order of kjv book
with open(f'../../src/routes/bible/roh.json.js') as f:
    bible = json.loads(f.read()[21:-67])
with open('Bible-kjv/bks.json') as f:
    bnames = json.load(f)

bible['identifier'] = 'kjv'
bible['name'] = 'King James Version'
for book in bible["books"]:
  try:
    book["aliases"].append(book["name"])
  except KeyError:
    book["aliases"] = [book["name"]]
  book['name'] = bnames[book['abbreviation']]

with open(f'../../src/routes/bible/kjv.json.js', 'w') as f:
  f.write('export const index = ')
  json.dump(bible, f, indent=2, ensure_ascii=False)
  f.write(""";

export async function get() {
  return {
    body: index,
  }
}""")

In [9]:
# Convert downloaded KJV
import json
with open('Bible-kjv/bks.json') as f:
    bnames = json.load(f)
bible = {}
for abbr, bname in bnames.items():
    with open(f"Bible-kjv/{bname.replace(' ','')}.json") as f:
        book = json.load(f)
    newbook = {}
    for ch in book["chapters"]:
        verses = [""]*len(ch["verses"])
        for v in ch["verses"]:
            verses[int(v["verse"])-1] = v["text"]
        newbook[ch["chapter"]] = verses
    with open(f'kjv/{abbr}.json', 'w') as f:
        json.dump(newbook, f, indent=2, ensure_ascii=False)

In [ ]:
# Extract aliases
from collections import defaultdict
aliases = defaultdict([])
bibles = {}
for version in 'seb sep ssv roh bot kjv'.split():
    with open(f'../../src/routes/bible/{version}.json.js', 'w') as f:
        bible = json.loads(f.read()[21:-67])
    bibleNew = {}
    for book in bible["books"]:
      aliases[book['abbreviation']].append(book["name"])
      for alias in book.get('aliases', []):
        if alias not in aliases[book['abbreviation']]:
          aliases[book['abbreviation']].append(alias)
      book['name'] = bnames[book['abbreviation']]
    bibles[version] = bibleNew

In [46]:
# Scrape bible.com
import requests
versionId = 509
res = requests.get(f"https://www.bible.com/api/bible/version/{versionId}")
bible = res.json()
print([book['usfm'] for book in bible['books']])
abbreviation = bible['abbreviation']
for book in bible['books']:
    for chapter in book['chapters']:
        usfm = f"{chapter['usfm']}.{abbreviation}"
        print(usfm)
        res = requests.get(f"https://www.bible.com/_next/data/SDuHIg5MDt8o_ULC4nbR7/cs/bible/{versionId}/{usfm}.json?versionId={versionId}&usfm={usfm}")
        if res.status_code != 200:
            print(res.status_code, res.content)
        data = res.json()
        html = data['pageProps']['chapterInfo']['content']
        print(html)
        exit

['GEN', 'EXO', 'LEV', 'NUM', 'DEU', 'JOS', 'JDG', 'RUT', '1SA', '2SA', '1KI', '2KI', '1CH', '2CH', 'EZR', 'NEH', 'EST', 'JOB', 'PSA', 'PRO', 'ECC', 'SNG', 'ISA', 'JER', 'LAM', 'EZK', 'DAN', 'HOS', 'JOL', 'AMO', 'OBA', 'JON', 'MIC', 'NAM', 'HAB', 'ZEP', 'HAG', 'ZEC', 'MAL', 'MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL', 'EPH', 'PHP', 'COL', '1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE', '2PE', '1JN', '2JN', '3JN', 'JUD', 'REV']
GEN.1.CSP
<div class="version vid509 iso6393ces" data-vid="509" data-iso6393="ces">
    <div class="book bkGEN">
        <div class="chapter ch1" data-usfm="GEN.1">
            <div class="label">1</div><div class="s"><span class="heading">Po&#269;&#225;tek </span></div><div class="p"><span class="verse v1" data-usfm="GEN.1.1"><span class="label">1</span><span class="content">Na po&#269;&#225;tku</span><span class="note f"><span class="label">#</span><span class=" body"><span class="ft">P&#345; 8,22&#8211;24; &#381; 93,2; 102,25&#8

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [9]:
import ast
from collections import defaultdict
from html.parser import HTMLParser
from os import listdir
import orjson
import requests

def yves_decode(b):
  i = 0
  out = bytearray()
  for i in range(0, len(b)-1, 2):
    out.append((0xFF & (b[(i + 1)])) >> 5 | (b[(i + 1)]) << 3 & 0xFF)
    out.append((0xFF & (b[i])) >> 5 | (b[i]) << 3 & 0xFF)
  if len(b) % 2 == 1:
    out.append((0xFF & (b[i])) >> 5 | (b[i]) << 3 & 0xFF)
  return out

class YvesParser(HTMLParser):
  def __init__(self):
    super().__init__()
    self.opened = False
    self.verses = defaultdict(str)
    
  def handle_starttag(self, tag, attrs):
    if tag == 'span':
      attrs = dict(attrs)
      if attrs.get('class') == 'content':
        self.opened = True
      usfm = attrs.get('data-usfm')
      if usfm:
        b, ch, v = usfm.split('.')
        self.usfm = int(v)

  def handle_endtag(self, tag):
    self.opened = False

  def handle_data(self, data):
    if self.opened:
      self.verses[self.usfm] += data

def yves_verses(filename):
  with open(filename, 'rb') as f:
    s = yves_decode(f.read()).decode()
    # print(s)
    parser = YvesParser()
    parser.feed(s)
    return parser.verses

usfm_books = ['GEN', 'EXO', 'LEV', 'NUM', 'DEU', 'JOS', 'JDG', 'RUT', '1SA', '2SA', '1KI', '2KI', '1CH', '2CH', 'EZR', 'NEH', 'EST', 'JOB', 'PSA', 'PRO', 'ECC', 'SNG', 'ISA', 'JER', 'LAM', 'EZK', 'DAN', 'HOS', 'JOL', 'AMO', 'OBA', 'JON', 'MIC', 'NAM', 'HAB', 'ZEP', 'HAG', 'ZEC', 'MAL', 'MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL', 'EPH', 'PHP', 'COL', '1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE', '2PE', '1JN', '2JN', '3JN', 'JUD', 'REV']
with open('../../src/routes/bible/sep.json.js') as f:
  biblia = ast.literal_eval(f.read()[21:-66])
  biblia_books = [b['abbreviation'] for b in biblia['books']]
  usfm2biblia = dict(zip(usfm_books, biblia_books))
  index_by_usfm = dict(zip(usfm_books, biblia['books']))
print(len(usfm_books), usfm_books)
print(len(biblia_books), biblia_books)
bibleId = 'csp'
versionId = 509
bible = requests.get(f'https://www.bible.com/api/bible/version/{versionId}').json()
# for usfmbook in listdir(f'{versionId}-4'):
for biblebook in bible['books']:
  usfmbook = biblebook['usfm']
  try:
    yvesfilenames = listdir(f'{versionId}-4/{usfmbook}')
  except NotADirectoryError as e:
    print(e)
    continue
  book = {}
  for yvesname in yvesfilenames:
    ch, ext = yvesname.split('.', 1)
    chapter = yves_verses(f'{versionId}-4/{usfmbook}/{yvesname}')
    book[ch] = list(chapter.values())
  abbr = usfm2biblia[usfmbook]
  b = index_by_usfm[usfmbook]
  b['aliases'] = b.get('aliases', [])
  b['aliases'].append(b['name'])
  b['name'] = biblebook['human']
  with open(f'{bibleId}/{abbr}.json', 'wb') as f:
    f.write(orjson.dumps(book, option=orjson.OPT_INDENT_2))
with open('../../src/routes/bible/csp.json.js', 'wb') as f:
  f.write(b'export const index = ')
  f.write(orjson.dumps(biblia, option=orjson.OPT_INDENT_2))
  f.write(b""";

export async function get() {
  return {
    body: index,
  }
}""")

66 ['GEN', 'EXO', 'LEV', 'NUM', 'DEU', 'JOS', 'JDG', 'RUT', '1SA', '2SA', '1KI', '2KI', '1CH', '2CH', 'EZR', 'NEH', 'EST', 'JOB', 'PSA', 'PRO', 'ECC', 'SNG', 'ISA', 'JER', 'LAM', 'EZK', 'DAN', 'HOS', 'JOL', 'AMO', 'OBA', 'JON', 'MIC', 'NAM', 'HAB', 'ZEP', 'HAG', 'ZEC', 'MAL', 'MAT', 'MRK', 'LUK', 'JHN', 'ACT', 'ROM', '1CO', '2CO', 'GAL', 'EPH', 'PHP', 'COL', '1TH', '2TH', '1TI', '2TI', 'TIT', 'PHM', 'HEB', 'JAS', '1PE', '2PE', '1JN', '2JN', '3JN', 'JUD', 'REV']
66 ['gn', 'ex', 'lv', 'nm', 'dt', 'joz', 'sdc', 'rut', '1sam', '2sam', '1krl', '2krl', '1krn', '2krn', 'ezd', 'neh', 'est', 'job', 'z', 'pris', 'koh', 'vlp', 'iz', 'jer', 'nar', 'ez', 'dan', 'oz', 'joel', 'am', 'abd', 'jon', 'mich', 'nah', 'hab', 'sof', 'ag', 'zach', 'mal', 'mt', 'mk', 'lk', 'jn', 'sk', 'rim', '1kor', '2kor', 'ga', 'ef', 'flp', 'kol', '1tes', '2tes', '1tim', '2tim', 'tit', 'flm', 'heb', 'jk', '1pt', '2pt', '1jn', '2jn', '3jn', 'jud', 'zj']
